In [3]:
import numpy as np
import pandas as pd
import itertools

In [4]:
characters = np.array(['A', 'C', 'G', 'T'])
characters_dict = dict((character_index, character) for character_index, character in enumerate(characters))

delta_table = np.array([
    (0,0,0,1),
    (0,0,1,0),
    (0,1,0,0),
    (1,0,0,0)
])

delta_dict = dict(((characters_dict[j],characters_dict[i]), int(delta_table[i][j])) for i in range(len(delta_table)) for j in range(len(delta_table[0])))

def delta(a, b):
    return delta_dict[a,b]

In [20]:
def nussinov(genome):
    n_table = np.empty((len(genome), len(genome)))
    # n_table.fill(np.nan)
    np.fill_diagonal(n_table, 0)

    diag_x, diag_y = np.diag_indices_from(n_table)
    diag_x = diag_x[diag_x > 0]
    diag_y = diag_y -1
    diag_y = diag_y[diag_y >= 0]

    n_table[(diag_x, diag_y)] = 0

    cells = itertools.product(range(n_table.shape[0]), range(n_table.shape[1]))

    for column, row in cells:
        if column > row:
            first = n_table[row+1, column-1] + delta(genome[row], genome[column])
            second = max([n_table[row, k] + n_table[k+1, column] for k in range(row, column)])

            n_table[row, column] = max(first, second)

    # pd.set_option('precision', 0)
    table_df = pd.DataFrame(n_table)
    
    # table_df = table_df.astype('int32')

    return table_df

In [21]:
genome = 'GCACGACG'

nussinov_table = nussinov(genome)
print(nussinov_table)

    0   1   2   3   4   5  6  7
0   0   1   1   1   2   2  2  3
1   0   0   0   0   1   1  1  2
2 NaN   0   0   0   1   1  1  2
3 NaN NaN   0   0   1   1  1  2
4 NaN NaN NaN   0   0   0  1  1
5 NaN NaN NaN NaN   0   0  0  1
6 NaN NaN NaN NaN NaN   0  0  1
7 NaN NaN NaN NaN NaN NaN  0  0


In [22]:
complementary_replacements = {"A": "T", "T": "A", "C": "G", "G": "C"}

def is_complementary(genes):
    return complementary_replacements[genes[0]] == genes[1]

In [40]:
def nussinov_traceback(i, j):
    if j <= i:
        return

    if nussinov_table.iloc[i][j] == nussinov_table.iloc[i][j-1]:
        nussinov_traceback(i, j-1)
    else:
        for k in range(i, j):
            if k >= 0 and \
                is_complementary((genome[k], genome[j])) and \
                nussinov_table.iloc[i][j] == (0 if k == 0 else nussinov_table.iloc[i][k-1]) + nussinov_table.iloc[k+1][j-1] + 1:
                print(f'({k+1}, {j+1})')
                nussinov_traceback(i, k-1)
                nussinov_traceback(k+1, j-1)
                return

In [42]:
nussinov_traceback(0, 7)

(4, 8)
(1, 2)
(5, 7)


![alt text](result.jpg "Vizualizace výsledné RNA sekvence")